In [ ]:
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import seaborn as sns

# Loading the data

Let's start by loading the data in pandas dataframes.

## Campaign desc

In [ ]:
df_campaign_desc = pd.read_csv('dunnhumby/campaign_desc.csv')

In [ ]:
df_campaign_desc.head()

In [ ]:
print('Number of campaign: ', len(df_campaign_desc.CAMPAIGN))
print('Unique description: ', df_campaign_desc.DESCRIPTION.unique())

Let's add a column with the duration of each campaign

In [ ]:
df_campaign_desc['DURATION'] = df_campaign_desc['END_DAY'] - df_campaign_desc['START_DAY'] 
df_campaign_desc.sort_values(by = ['DESCRIPTION', 'DURATION'])

## Campaign table

In [ ]:
df_campaign_table = pd.read_csv('dunnhumby/campaign_table.csv')

In [ ]:
df_campaign_table.head()

Let's study the distribution of the number of campaigns each household has benefited from.

In [ ]:
nbins = df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().max()
df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().hist(bins = nbins)
plt.title('Distribution of the number of campaigns per household')
plt.xlabel('Number of campaigns')
plt.ylabel('Number of households')
plt.show()

In [ ]:
print('Median number of campaigns:', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().median())
print('Max number of campaigns:', df_campaign_table.groupby(by = 'household_key').CAMPAIGN.count().max())

## Causal data

In [ ]:
df_causal = pd.read_csv('dunnhumby/causal_data.csv')

In [ ]:
df_causal.head()

## Coupon redemption

In [ ]:
df_coupon_redempt = pd.read_csv('dunnhumby/coupon_redempt.csv')

In [ ]:
df_coupon_redempt.head()

## Coupon

In [ ]:
df_coupon = pd.read_csv('dunnhumby/coupon.csv')

In [ ]:
df_coupon.head()

## Demographics

In [ ]:
df_demo = pd.read_csv('dunnhumby/hh_demographic.csv')

In [ ]:
df_demo.head()

## Products

In [ ]:
df_products = pd.read_csv('dunnhumby/product.csv')

In [ ]:
df_products.head()

## Transcation data

In [ ]:
df_transaction = pd.read_csv('dunnhumby/transaction_data.csv')

In [ ]:
df_transaction.head()